# Stablecoin Premiums — Quick Start

This notebook demonstrates how to fetch P2P stablecoin quotes, reference FX rates, and compute premiums/spreads using the `stablecoin_premiums` package.

Prerequisites:
- Create and activate a virtual environment
- `pip install -r requirements.txt`
- Copy `.env.example` to `.env` and fill credentials (especially XE API credentials)
- Never commit `.env` to version control

Notes:
- The output `../data/` directory is gitignored and safe for local exports.
- You can add or swap providers under `src/stablecoin_premiums/clients/` as needed.

In [ ]:
# Optional: load a local .env file (best-effort). If python-dotenv is not installed, this is skipped.
try:
    from dotenv import load_dotenv  # type: ignore
    load_dotenv()
except Exception:
    pass

from stablecoin_premiums import setup_logging
setup_logging("INFO")

print("Environment loaded (if present). Logging initialized.")

## Helpers and Imports

We import the Binance P2P client for quotes, the XE client for FX rates, and use `compute_premiums` to calculate the metrics. 

If you prefer different providers, add adapters under `stablecoin_premiums/clients/` and swap them in here.

In [ ]:
from typing import Optional, Dict, Any

import pandas as pd

from stablecoin_premiums.clients.binance import average_price
from stablecoin_premiums.clients.xe import fetch_fx_rate
from stablecoin_premiums.compute import compute_premiums

def fetch_premium_row(
    fiat: str,
    *,
    asset: str = "USDT",
    ref_fiat: str = "USD",
    decimals: Optional[int] = 2,
    min_valid_ads: Optional[int] = None,
) -> Dict[str, Any]:
    """
    Fetch BUY/SELL quotes for a fiat (Binance P2P), get FX mid (XE),
    and compute premiums/spreads.

    Returns a dict with fields:
      - fiat, asset, ref_fiat
      - buy_rate, sell_rate
      - fx_bid, fx_ask (if FX available)
      - stablecoin_sell_premium, stablecoin_buy_premium, stablecoin_buy_sell_spread (if computable)
      - status: "ok" or error marker
    """
    row: Dict[str, Any] = {"fiat": fiat, "asset": asset, "ref_fiat": ref_fiat}

    buy = average_price(fiat, asset, "BUY", min_valid_ads=min_valid_ads)
    sell = average_price(fiat, asset, "SELL", min_valid_ads=min_valid_ads)
    fx = fetch_fx_rate(fiat, ref_fiat)

    row["buy_rate"] = buy
    row["sell_rate"] = sell

    if buy is None or sell is None:
        row["status"] = "insufficient_p2p_data"
        return row

    if not fx:
        row["status"] = "insufficient_fx_data"
        return row

    row["fx_bid"] = fx["bid"]
    row["fx_ask"] = fx["ask"]

    try:
        premiums = compute_premiums(
            sell_rate=sell,
            buy_rate=buy,
            fx_bid=fx["bid"],
            fx_ask=fx["ask"],
            decimals=decimals,
        )
        row.update(premiums.to_dict())
        row["status"] = "ok"
    except Exception as exc:
        row["status"] = f"compute_error: {exc}"

    return row

print("Helper ready: fetch_premium_row(fiat, asset='USDT', ref_fiat='USD')")

## Run for a Few Markets

Adjust the `fiats` list as needed. You can also pass `min_valid_ads` to enforce a minimum number of valid P2P ads for averaging.

In [ ]:
fiats = ["MXN", "BRL", "ARS"]  # Edit this list to your target fiats
rows = [fetch_premium_row(f, asset="USDT", ref_fiat="USD", decimals=2, min_valid_ads=None) for f in fiats]
df = pd.DataFrame(rows)
df

## Save Results (Optional)

Export results to a CSV in the `../data/` directory (gitignored by default).

In [ ]:
from datetime import datetime
import os

os.makedirs("../data", exist_ok=True)
outfile = f"../data/premiums_{datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}.csv"
df.to_csv(outfile, index=False)
outfile

## JSON Output (Optional)

Pretty-print the results as JSON (useful for programmatic usage or storing snapshots).

In [ ]:
import json
print(json.dumps(rows, indent=2, ensure_ascii=False))

## Next Steps

- Add more providers or adjust validation rules under `src/stablecoin_premiums/clients/`
- Schedule periodic runs (e.g., cron + CLI) and write to a data store
- Visualize time-series of premiums in additional notebooks
- Add tests under `tests/` for reproducible CI
- Contribute improvements via pull requests!